<a href="https://colab.research.google.com/github/Acefite/Projet_SIG/blob/main/TDFred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Etude des liens entre les naufrages de navires
Rendu De Charles Champion 086

Les naufrages font partie intégrante de l’histoire maritime. Qu’ils résultent de conditions météorologiques extrêmes, d’erreurs humaines, de conflits armés ou de défaillances structurelles.

Cette étude vise à analyser un ensemble de données sur les naufrages afin d’identifier les facteurs qui peuvent aggraver le risque ou la gravité de ces événements. Parmi les paramètres explorés figurent notamment le type de navire, les matériaux utilisés pour la construction, la période historique — incluant des moments critiques comme les guerres mondiales — ainsi que d’autres caractéristiques susceptibles d’influencer la survenue ou l’issue d’un naufrage.

L’objectif est ainsi de mieux comprendre les dynamiques derrière ces pertes maritimes et de dégager des tendances pertinentes pour l’analyse historique, technique ou sécuritaire.

In [1]:
!pip install geodatasets

Connection Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Premières manipulations

Import du csv

In [3]:
import pandas as pd

df = pd.read_excel('/content/drive/MyDrive/ULS/CSV_Shipwreck_Database_120924_SR.csv.xlsx')

df.head()

,SHIPS NAME,AKA,SHIPS OWNER,VESSEL TYPE,YEAR BUILT,WHERE BUILT,DATE LOST,YEAR,MNTH,DAY,...,NUM PASS,LIVES LOST,SHIP VALUE,CARGO VALUE,NATURE OF CARGO,USLSS STATION NAME,MAP,LOST,PHOTO ON FILE,MISC INFORMATION
0,74 H,120924,NaN,Scow,NaN,NaN,7/28/1916,1916.0,7,28,...,NaN,0,NaN,NaN,NaN,USCG #97 & 99,NaN,NaN,N,NaN
1,A B Crosby,NaN,NaN,Schooner,1884,NaN,3/12/1888,1888.0,3,12,...,NaN,0,NaN,NaN,Coconuts,NaN,N,N,N,Refloated; Made New York 3/26/1888
2,A B Sherman,NaN,NaN,Schooner,1883,NaN,12/8/1886,1886.0,12,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,#106238
3,A B Thompson,NaN,NaN,Yacht,NaN,NaN,7/18/1874,1874.0,7,18,...,7,5,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
4,A C Austin,NaN,NaN,Schooner,NaN,NaN,2/1868,1868.0,2,NaN,...,NaN,NaN,30000,NaN,Coal,NaN,NaN,NaN,N,NaN


In [4]:
df.shape

(4600, 36)

On retire les navires sur lesquels on a pas d'informations sur la latitude et la longitude.

In [5]:
df.columns
# On vérifie le nom des colonnes latitudes et longitudes

Index(['SHIPS NAME', 'AKA', 'SHIPS OWNER', 'VESSEL TYPE', 'YEAR BUILT',
       'WHERE BUILT', 'DATE LOST', 'YEAR', 'MNTH', 'DAY', 'LOCATION LOST',
       'LATITUDE', 'LONGITUDE', 'CAUSE OF LOSS', 'CONSTRUCTION', 'FLAG',
       'LENGTH', 'BEAM', 'DRAFT', 'GROSS TONNAGE', 'NET TONNAGE',
       'HOME HAILING PORT', 'DEPARTURE PORT', 'DESTINATION PORT', 'MASTER',
       'NUM CREW', 'NUM PASS', 'LIVES LOST', 'SHIP VALUE', 'CARGO VALUE',
       'NATURE OF CARGO', 'USLSS STATION NAME', 'MAP', 'LOST', 'PHOTO ON FILE',
       'MISC INFORMATION'],
      dtype='object')

In [6]:
df = df.dropna(subset=['LATITUDE', 'LONGITUDE'])
df.head()


,SHIPS NAME,AKA,SHIPS OWNER,VESSEL TYPE,YEAR BUILT,WHERE BUILT,DATE LOST,YEAR,MNTH,DAY,...,NUM PASS,LIVES LOST,SHIP VALUE,CARGO VALUE,NATURE OF CARGO,USLSS STATION NAME,MAP,LOST,PHOTO ON FILE,MISC INFORMATION
21,A M C Smith,NaN,NaN,Schooner,NaN,NaN,10/13/1876,1876.0,10,13,...,NaN,0,4000,NaN,NaN,Green Island #12,A,N,N,#1711; Most reports say total loss
26,A T Coleman,A J Coleman; Sallie,NaN,Schooner,1865,"Talbot City, MD",1/20/1901,1901.0,1,20,...,0,0,1800,800,Lumber,Atlantic City,A,Y,N,#22033; Total loss
39,Aberdeen,NaN,S H Mills,Bark,1856,"Searsport, ME",1/6/1885,1885.0,1,6,...,0,0,7000,38000,Cotton,"Harvey Cedars, Loveladies, & Ship Bottom",A,Y,N,#218; Total loss
40,Abiel Abbot,NaN,Abiel Abbot,Bark,1874,"Boston, MA",1/20/1903,1903.0,1,20,...,0,5,8000,3000,Salt,Long Beach Island & Ship Bottom,A,Y,N,#105479; Total loss
46,Acara,NaN,J M Wood,Screw schooner,1898,"Newcastle, England",3/1/1902,1902.0,3,1,...,1,0,NaN,600000,"Tin Ingots, tea, spices",NaN,A,Y,Y,Broke up in 3 days; Total loss


In [7]:
df.shape

(739, 36)

On travaille maintenant avec 739 navires dont on connait la position géographique du naufrages à la place de travailler avec 4600 navires. Un autre avantage est que ça allège la base de données et les calculs devraient être plus rapide.

On vérifie le type des colonnes latitudes et longitudes

In [8]:
df[['LATITUDE', 'LONGITUDE']].dtypes

,0
LATITUDE,object
LONGITUDE,object


Ils sont de type objets il faut donc les convertir au format numérique pour éviter que ça pose problème par la suite.

In [9]:
df['LATITUDE'] = pd.to_numeric(df['LATITUDE'], errors='coerce')
df['LONGITUDE'] = pd.to_numeric(df['LONGITUDE'], errors='coerce')

On supprime les lignes où la conversion entraine des coordonnées non valides (par exemple NaN)

In [10]:
df = df.dropna(subset=['LATITUDE', 'LONGITUDE'])

In [11]:
print(df[['LATITUDE', 'LONGITUDE']].dtypes)
print(df.shape)

LATITUDE     float64
LONGITUDE    float64
dtype: object
(714, 36)


# Création d'intervalles de temps pour l'étude


Pour l'étude et suivre l'évolution des facteurs aggravant lors des naufrages au cours du temps, on va créer des intervalles de 25 ans qui commencent en 1900.

Parmis les causes de naufrages, on peut observer des navires qui ont été torpillés, probablement à cause des guerres. Il est donc intéressant de rajouter des intervalles pour les deux guerres mondiales pour vérifier ce fait.

In [12]:
df['DATE LOST'] = pd.to_datetime(df['DATE LOST'], format='%d/%m/%Y', errors='coerce') # On précise le format des dates
df['YEAR'] = df['DATE LOST'].dt.year
df = df.dropna(subset=['YEAR'])
df['YEAR'] = df['YEAR'].astype(int)
# Tout d'abord on vérifie qu'il ne manque pas de date et on prend les années

In [13]:
def get_period(y):
    # Périodes des guerres
    if 1914 <= y <= 1918:
        return "WWI (1914–1918)"
    if 1939 <= y <= 1945:
        return "WWII (1939–1945)"

    # Périodes générales
    if y < 1900:
        return "Avant 1900"
    elif 1900 <= y < 1925:
        return "1900–1925"
    elif 1925 <= y < 1950:
        return "1925–1950"
    elif 1950 <= y < 1975:
        return "1950–1975"
    elif 1975 <= y < 2000:
        return "1975–2000"
    elif 2000 <= y:
        return "2000–2025"

In [14]:
df['PERIODE'] = df['YEAR'].apply(get_period)

In [15]:
df['PERIODE'].value_counts().sort_index()

,count
PERIODE,
1900–1925,24
1925–1950,24
1950–1975,18
1975–2000,3
2000–2025,2
Avant 1900,127
WWI (1914–1918),23
WWII (1939–1945),24


In [16]:
df.shape

(245, 37)

On voit que le nombre de lignes diminue encore, cela est dû au fait que le format JJ/MM/AAAA n'est pas appliqué à toute les lignes. On a maintenant des lignes avec un format de date homogène.

# Analyse par catégories

On compte les différents classes de navire, les différentes causes de naufrages et le nombre de navires construits avec certains type matériaux (bois, acier...)

## Type de Navire

In [17]:
# On remplace les valeurs manquantes / vides par Unknwon si il y en a
df['VESSEL TYPE'] = df['VESSEL TYPE'].fillna('Unknown')
df['VESSEL TYPE'] = df['VESSEL TYPE'].replace([''], 'Unknown')

In [18]:
df['VESSEL TYPE'].value_counts()
# Type de navire

,count
VESSEL TYPE,
Schooner,72
Barge,18
Steamship,15
Sloop,13
Bark,12
Freighter,10
Brig,10
Schooner - Barge,9
Schooner 3 masted,8


Comme il y a des catégories avec qu'un seul représentant on va les regrouper en une catégorie (autre)

In [19]:
threshold = 3 # Correspond à un seuil qui est strictement inférieur à 3 (<3)
other = df['VESSEL TYPE'].value_counts()[df['VESSEL TYPE'].value_counts() < threshold].index # On prend les types qui ont moins de 2 représentants
df['VESSEL TYPE'] = df['VESSEL TYPE'].replace(other, 'Other')
df['VESSEL TYPE'].value_counts()

,count
VESSEL TYPE,
Schooner,72
Other,51
Barge,18
Steamship,15
Sloop,13
Bark,12
Brig,10
Freighter,10
Schooner - Barge,9


## Matériaux

In [20]:
# On remplace les valeurs manquantes / vides par Unknown si il y en a
df['CONSTRUCTION'] = df['CONSTRUCTION'].fillna('Unknown')
df['CONSTRUCTION'] = df['CONSTRUCTION'].replace([''], 'Unknown')

In [21]:
df['CONSTRUCTION'].value_counts()
# Matériaux

,count
CONSTRUCTION,
Unknown,148
Wood,60
Steel,30
Iron,4
Steel,1
"Oak, cedar",1
Fiberglass,1


Comme il y a des catégories avec qu'un seul représentant on va les regrouper en une catégorie (autre)

In [22]:
threshold = 3 # Correspond à un seuil qui est strictement inférieur à 3 (<3)
other = df['CONSTRUCTION'].value_counts()[df['CONSTRUCTION'].value_counts() < threshold].index # On prend les types qui ont moins de 2 représentants
df['CONSTRUCTION'] = df['CONSTRUCTION'].replace(other, 'Other')
df['CONSTRUCTION'].value_counts()

,count
CONSTRUCTION,
Unknown,148
Wood,60
Steel,30
Iron,4
Other,3


## Cause du Naufrage

In [23]:
# On remplace les valeurs manquantes / vides par Unknwon si il y en a
df['CAUSE OF LOSS'] = df['CAUSE OF LOSS'].fillna('Unknown')
df['CAUSE OF LOSS'] = df['CAUSE OF LOSS'].replace([''], 'Unknown')

In [24]:
df['CAUSE OF LOSS'].value_counts()
# Cause de naufrage

,count
CAUSE OF LOSS,
Unknown,64
Stranded,46
Foundered,17
Grounded,15
Sank,7
...,...
Stranded in snow storm,1
Destroyed by British,1
Torpedoed by U-103; War loss,1


Comme il y a des catégories avec qu'un seul représentant on va les regrouper en une catégorie (autre)

In [25]:
threshold = 3 # Correspond à un seuil qui est strictement inférieur à 3 (<3)
other = df['CAUSE OF LOSS'].value_counts()[df['CAUSE OF LOSS'].value_counts() < threshold].index # On prend les types qui ont moins de 2 représentants
df['CAUSE OF LOSS'] = df['CAUSE OF LOSS'].replace(other, 'Other')
df['CAUSE OF LOSS'].value_counts()

,count
CAUSE OF LOSS,
Unknown,64
Other,61
Stranded,46
Foundered,17
Grounded,15
Sank,7
Wrecked,7
Stranded in storm,6
Burned,6


In [26]:
df

,SHIPS NAME,AKA,SHIPS OWNER,VESSEL TYPE,YEAR BUILT,WHERE BUILT,DATE LOST,YEAR,MNTH,DAY,...,LIVES LOST,SHIP VALUE,CARGO VALUE,NATURE OF CARGO,USLSS STATION NAME,MAP,LOST,PHOTO ON FILE,MISC INFORMATION,PERIODE
39,Aberdeen,NaN,S H Mills,Bark,1856,"Searsport, ME",1885-06-01,1885,1,6,...,0,7000,38000,Cotton,"Harvey Cedars, Loveladies, & Ship Bottom",A,Y,N,#218; Total loss,Avant 1900
46,Acara,NaN,J M Wood,Other,1898,"Newcastle, England",1902-01-03,1902,3,1,...,0,NaN,600000,"Tin Ingots, tea, spices",NaN,A,Y,Y,Broke up in 3 days; Total loss,1900–1925
52,Ada F Crosby,NaN,John Baizley & Sons,Bark,1875,"Calais, ME",1881-05-10,1881,10,5,...,1,42000,3500,819 ton Coal,NaN,F,Y,N,#105648; Total loss; Rescued by Mary C Haskell,Avant 1900
59,Addie P McFadden,NaN,NaN,Schooner,1890,"Bath, ME",1894-10-04,1894,4,10,...,0,14000,300,Ice,Ocean City,A,NaN,N,#106750,Avant 1900
79,Adolphus,NaN,Abel C Robbins,Other,1873,"Green Cove, NS",1886-04-01,1886,1,4,...,0,15000,5000,"Barrels, chalk",Lewes; Cape Henlopen,A,Y,N,#66708; Total loss,Avant 1900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4554,Winneconne,NaN,Ammerican Transatlantic Co,Freighter,1907,"Sunderland, England",1918-02-06,1918,6,2,...,NaN,NaN,NaN,NaN,NaN,F,Y,Y,#213661; Hydro #842; Black Sundaay wreck,WWI (1914–1918)
4570,Yankee,NaN,U S Shipping Board,Freighter steam,1890,"Cleveland, OH",1919-11-06,1919,6,11,...,0,NaN,NaN,Coal,NaN,F,Y,Y,#86129; Hydro Office #589,1900–1925
4581,YRB - 9,NaN,NaN,Barge,NaN,NaN,1956-12-06,1956,6,12,...,NaN,NaN,NaN,NaN,NaN,A,Y,N,NaN,1950–1975
4592,Zetland,NaN,C D Snow,Brig,1872,Nova Scotia,1881-02-11,1881,11,2,...,0,10000,5000,Salt,Beasleys #30 & Pecks #31,A,Y,N,#66732; Total loss,Avant 1900


# Création de la carte interactive

## Séparation par intervalle de temps

In [27]:
import folium

#On centre la carte là où la plupart des naufrages on eu lieu
m = folium.Map(location=[df['LATITUDE'].mean(), df['LONGITUDE'].mean()], zoom_start=3)

# On récupère les dates existantes (uniques)
periodes = df['PERIODE'].unique()

# Boucle pour créer un layer par période
for periode in periodes:
    layer = folium.FeatureGroup(name=periode)   # un layer = une période

    subset = df[df['PERIODE'] == periode]

    for _, row in subset.iterrows():
        popup_text = (
            f"<b>Nom :</b> {row['SHIPS NAME']}<br>"
            f"<b>Type :</b> {row['VESSEL TYPE']}<br>"
            f"<b>Matériau :</b> {row['CONSTRUCTION']}<br>"
            f"<b>Cause :</b> {row['CAUSE OF LOSS']}"
        )

        folium.CircleMarker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=4,
            popup=popup_text,
            color="blue",
            fill=True,
            fill_opacity=0.7
        ).add_to(layer)

    m.add_child(layer)

# On ajoute le contrôle des layers
folium.LayerControl().add_to(m)


m

On observe que l'ensemble des naufrages recensés avec une date et une localisation connues sont situés dans le long de l'Etat du New Jersey. Cela peut être dû au fait que le museum d'où provient le dataset se situe dans le même état. Donc ils ont sûrement plus d'informations précises sur les naufrages à proximité.

In [28]:
m.save("carte_date.html")

## Rajout des catégories

Le code est identique pour toute les périodes il y a juste les dates qui changent.

On va créer une carte avec les layers des catégories (causes de naufrages, matériaux et types de navires) pour chaque période qu'on a déterminées.

In [29]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

## Avant 1900

In [30]:
# On sélectionne les dates
periode = "Avant 1900"
df_periode = df[df["PERIODE"] == periode].copy()

# On définit les catégories qu'on voudra afficher dans les différents layers
categories = {
    "Materials (CONSTRUCTION)": "CONSTRUCTION",
    "Ship Type (VESSEL TYPE)": "VESSEL TYPE",
    "Cause of Loss": "CAUSE OF LOSS"
}


m_avant_1900 = folium.Map(
    location=[df_periode["LATITUDE"].mean(), df_periode["LONGITUDE"].mean()],
    zoom_start=4
)

# On crée les couleurs qui seront utilisées dans la légende
def make_color_dict(values):
    unique_vals = sorted(values.unique())
    cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
    return {cat: colors.rgb2hex(cmap(i)) for i, cat in enumerate(unique_vals)}


legends = ""

# On crée les layers et les légendes
for layer_name, col in categories.items():

    layer = folium.FeatureGroup(name=layer_name, show=False)

    color_dict = make_color_dict(df_periode[col])

    # On ajoute les points selon les coordonnées
    for _, row in df_periode.iterrows():
        color = color_dict[row[col]]
        folium.CircleMarker(
            location=[row["LATITUDE"], row["LONGITUDE"]],
            radius=4,
            popup=f"<b>{row['SHIPS NAME']}</b><br>{col}: {row[col]}",
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        ).add_to(layer)

    m_avant_1900.add_child(layer)

    # On crée la légende dans une boîte en bas à gauche de la carte (c'est en html)
    legend_html = f"""
    <div id="legend_{layer_name.replace(' ', '_')}"
         style="display:none;
                position: fixed;
                bottom: 30px;
                left: 30px;
                z-index:9999;
                background-color: white;
                border:2px solid grey;
                padding:10px;
                font-size:13px;">
    <b>{layer_name}</b><br>
    """

    for cat, col_hex in color_dict.items():
        legend_html += f"""
        <i style="background:{col_hex};
                  width:12px;height:12px;
                  float:left;margin-right:6px;"></i>{cat}<br>
        """

    legend_html += "</div>"
    legends += legend_html

# Ajouter les légendes à la carte
m_avant_1900.get_root().html.add_child(folium.Element(legends))

# Script pour que ce soit la bonne légende qui s'affiche en fonction du layer
# le script est en java (ça marche car folium se base sur leaflet qui est une bibliothèque de java)
js = """
<script>

function hideLegends() {
    document.querySelectorAll('[id^=legend_]').forEach(el => {
        el.style.display = "none";
    });
}

// Exécuter au chargement
hideLegends();

// Intercepter les clics dans le LayerControl
document.addEventListener("change", function(e) {
    if (e.target.classList.contains("leaflet-control-layers-selector")) {

        hideLegends();

        if (e.target.checked) {
            // retrouver le nom du layer dans le label voisin
            let label = e.target.nextSibling.innerText.trim();
            let id = "legend_" + label.replaceAll(" ", "_");
            let legendEl = document.getElementById(id);
            if (legendEl) { legendEl.style.display = "block"; }
        }
    }
});

</script>
"""

m_avant_1900.get_root().html.add_child(folium.Element(js))

# Ajouter LayerControl
folium.LayerControl().add_to(m_avant_1900)

m_avant_1900


/tmp/ipython-input-4270516250.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
/tmp/ipython-input-4270516250.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
/tmp/ipython-input-4270516250.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', l

In [31]:
m_avant_1900.save("carte_avant_1900.html")

## 1900-1925

In [32]:
# On sélectionne les dates
periode = "1900–1925"
df_periode = df[df["PERIODE"] == periode].copy()

# On définit les catégories qu'on voudra afficher dans les différents layers
categories = {
    "Materials (CONSTRUCTION)": "CONSTRUCTION",
    "Ship Type (VESSEL TYPE)": "VESSEL TYPE",
    "Cause of Loss": "CAUSE OF LOSS"
}


m_1900_1925 = folium.Map(
    location=[df_periode["LATITUDE"].mean(), df_periode["LONGITUDE"].mean()],
    zoom_start=4
)

# On crée les couleurs qui seront utilisées dans la légende
def make_color_dict(values):
    unique_vals = sorted(values.unique())
    cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
    return {cat: colors.rgb2hex(cmap(i)) for i, cat in enumerate(unique_vals)}


legends = ""

# On crée les layers et les légendes
for layer_name, col in categories.items():

    layer = folium.FeatureGroup(name=layer_name, show=False)

    color_dict = make_color_dict(df_periode[col])

    # On ajoute les points selon les coordonnées
    for _, row in df_periode.iterrows():
        color = color_dict[row[col]]
        folium.CircleMarker(
            location=[row["LATITUDE"], row["LONGITUDE"]],
            radius=4,
            popup=f"<b>{row['SHIPS NAME']}</b><br>{col}: {row[col]}",
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        ).add_to(layer)

    m_1900_1925.add_child(layer)

    # On crée la légende dans une boîte en bas à gauche de la carte (c'est en html)
    legend_html = f"""
    <div id="legend_{layer_name.replace(' ', '_')}"
         style="display:none;
                position: fixed;
                bottom: 30px;
                left: 30px;
                z-index:9999;
                background-color: white;
                border:2px solid grey;
                padding:10px;
                font-size:13px;">
    <b>{layer_name}</b><br>
    """

    for cat, col_hex in color_dict.items():
        legend_html += f"""
        <i style="background:{col_hex};
                  width:12px;height:12px;
                  float:left;margin-right:6px;"></i>{cat}<br>
        """

    legend_html += "</div>"
    legends += legend_html

# Ajouter les légendes à la carte
m_1900_1925.get_root().html.add_child(folium.Element(legends))

# Script pour que ce soit la bonne légende qui s'affiche en fonction du layer
# le script est en java (ça marche car folium se base sur leaflet qui est une bibliothèque de java)
js = """
<script>

function hideLegends() {
    document.querySelectorAll('[id^=legend_]').forEach(el => {
        el.style.display = "none";
    });
}

// Exécuter au chargement
hideLegends();

// Intercepter les clics dans le LayerControl
document.addEventListener("change", function(e) {
    if (e.target.classList.contains("leaflet-control-layers-selector")) {

        hideLegends();

        if (e.target.checked) {
            // retrouver le nom du layer dans le label voisin
            let label = e.target.nextSibling.innerText.trim();
            let id = "legend_" + label.replaceAll(" ", "_");
            let legendEl = document.getElementById(id);
            if (legendEl) { legendEl.style.display = "block"; }
        }
    }
});

</script>
"""

m_1900_1925.get_root().html.add_child(folium.Element(js))

# Ajouter LayerControl
folium.LayerControl().add_to(m_1900_1925)

m_1900_1925


/tmp/ipython-input-61846726.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
/tmp/ipython-input-61846726.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
/tmp/ipython-input-61846726.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(uni

In [33]:
m_1900_1925.save("carte_1900_1925.html")

## 1925-1950

In [34]:
# On sélectionne les dates
periode = "1925–1950"
df_periode = df[df["PERIODE"] == periode].copy()

# On définit les catégories qu'on voudra afficher dans les différents layers
categories = {
    "Materials (CONSTRUCTION)": "CONSTRUCTION",
    "Ship Type (VESSEL TYPE)": "VESSEL TYPE",
    "Cause of Loss": "CAUSE OF LOSS"
}


m_1925_1950 = folium.Map(
    location=[df_periode["LATITUDE"].mean(), df_periode["LONGITUDE"].mean()],
    zoom_start=4
)

# On crée les couleurs qui seront utilisées dans la légende
color_dict = make_color_dict(df_periode[col])


legends = ""

# On crée les layers et les légendes
for layer_name, col in categories.items():

    layer = folium.FeatureGroup(name=layer_name, show=False)

    vals = df_periode[col].dropna().unique()
    if len(vals) == 0:
        print(f"Aucune donnée pour {col} dans la période {periode}, on saute ce layer.")
        continue  # saute la catégorie si vide
    color_dict = make_color_dict(df_periode[col].dropna())


    # On ajoute les points selon les coordonnées
    for _, row in df_periode.iterrows():
        color = color_dict[row[col]]
        folium.CircleMarker(
            location=[row["LATITUDE"], row["LONGITUDE"]],
            radius=4,
            popup=f"<b>{row['SHIPS NAME']}</b><br>{col}: {row[col]}",
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        ).add_to(layer)

    m_1925_1950.add_child(layer)

    # On crée la légende dans une boîte en bas à gauche de la carte (c'est en html)
    legend_html = f"""
    <div id="legend_{layer_name.replace(' ', '_')}"
         style="display:none;
                position: fixed;
                bottom: 30px;
                left: 30px;
                z-index:9999;
                background-color: white;
                border:2px solid grey;
                padding:10px;
                font-size:13px;">
    <b>{layer_name}</b><br>
    """

    for cat, col_hex in color_dict.items():
        legend_html += f"""
        <i style="background:{col_hex};
                  width:12px;height:12px;
                  float:left;margin-right:6px;"></i>{cat}<br>
        """

    legend_html += "</div>"
    legends += legend_html

# Ajouter les légendes à la carte
m_1925_1950.get_root().html.add_child(folium.Element(legends))

# Script pour que ce soit la bonne légende qui s'affiche en fonction du layer
# le script est en java (ça marche car folium se base sur leaflet qui est une bibliothèque de java)
js = """
<script>

function hideLegends() {
    document.querySelectorAll('[id^=legend_]').forEach(el => {
        el.style.display = "none";
    });
}

// Exécuter au chargement
hideLegends();

// Intercepter les clics dans le LayerControl
document.addEventListener("change", function(e) {
    if (e.target.classList.contains("leaflet-control-layers-selector")) {

        hideLegends();

        if (e.target.checked) {
            // retrouver le nom du layer dans le label voisin
            let label = e.target.nextSibling.innerText.trim();
            let id = "legend_" + label.replaceAll(" ", "_");
            let legendEl = document.getElementById(id);
            if (legendEl) { legendEl.style.display = "block"; }
        }
    }
});

</script>
"""

m_1925_1950.get_root().html.add_child(folium.Element(js))

# Ajouter LayerControl
folium.LayerControl().add_to(m_1925_1950)

m_1925_1950


/tmp/ipython-input-61846726.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
/tmp/ipython-input-61846726.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
/tmp/ipython-input-61846726.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(uni

In [35]:
m_1925_1950.save("carte_1925_1950.html")

## 1950-1975

In [36]:
# On sélectionne les dates
periode = "1950–1975"
df_periode = df[df["PERIODE"] == periode].copy()

# On définit les catégories qu'on voudra afficher dans les différents layers
categories = {
    "Materials (CONSTRUCTION)": "CONSTRUCTION",
    "Ship Type (VESSEL TYPE)": "VESSEL TYPE",
    "Cause of Loss": "CAUSE OF LOSS"
}


m_1950_1975 = folium.Map(
    location=[df_periode["LATITUDE"].mean(), df_periode["LONGITUDE"].mean()],
    zoom_start=4
)

# On crée les couleurs qui seront utilisées dans la légende
def make_color_dict(values):
    unique_vals = sorted(values.unique())
    cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
    return {cat: colors.rgb2hex(cmap(i)) for i, cat in enumerate(unique_vals)}


legends = ""

# On crée les layers et les légendes
for layer_name, col in categories.items():

    layer = folium.FeatureGroup(name=layer_name, show=False)

    color_dict = make_color_dict(df_periode[col])

    # On ajoute les points selon les coordonnées
    for _, row in df_periode.iterrows():
        color = color_dict[row[col]]
        folium.CircleMarker(
            location=[row["LATITUDE"], row["LONGITUDE"]],
            radius=4,
            popup=f"<b>{row['SHIPS NAME']}</b><br>{col}: {row[col]}",
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        ).add_to(layer)

    m_1950_1975.add_child(layer)

    # On crée la légende dans une boîte en bas à gauche de la carte (c'est en html)
    legend_html = f"""
    <div id="legend_{layer_name.replace(' ', '_')}"
         style="display:none;
                position: fixed;
                bottom: 30px;
                left: 30px;
                z-index:9999;
                background-color: white;
                border:2px solid grey;
                padding:10px;
                font-size:13px;">
    <b>{layer_name}</b><br>
    """

    for cat, col_hex in color_dict.items():
        legend_html += f"""
        <i style="background:{col_hex};
                  width:12px;height:12px;
                  float:left;margin-right:6px;"></i>{cat}<br>
        """

    legend_html += "</div>"
    legends += legend_html

# Ajouter les légendes à la carte
m_1950_1975.get_root().html.add_child(folium.Element(legends))

# Script pour que ce soit la bonne légende qui s'affiche en fonction du layer
# le script est en java (ça marche car folium se base sur leaflet qui est une bibliothèque de java)
js = """
<script>

function hideLegends() {
    document.querySelectorAll('[id^=legend_]').forEach(el => {
        el.style.display = "none";
    });
}

// Exécuter au chargement
hideLegends();

// Intercepter les clics dans le LayerControl
document.addEventListener("change", function(e) {
    if (e.target.classList.contains("leaflet-control-layers-selector")) {

        hideLegends();

        if (e.target.checked) {
            // retrouver le nom du layer dans le label voisin
            let label = e.target.nextSibling.innerText.trim();
            let id = "legend_" + label.replaceAll(" ", "_");
            let legendEl = document.getElementById(id);
            if (legendEl) { legendEl.style.display = "block"; }
        }
    }
});

</script>
"""

m_1950_1975.get_root().html.add_child(folium.Element(js))

# Ajouter LayerControl
folium.LayerControl().add_to(m_1950_1975)

m_1950_1975


/tmp/ipython-input-3403423859.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
/tmp/ipython-input-3403423859.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
/tmp/ipython-input-3403423859.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', l

In [37]:
m_1950_1975.save("carte_1950_1975.html")

## 1975-2000

In [38]:
# On sélectionne les dates
periode = "1975–2000"
df_periode = df[df["PERIODE"] == periode].copy()

# On définit les catégories qu'on voudra afficher dans les différents layers
categories = {
    "Materials (CONSTRUCTION)": "CONSTRUCTION",
    "Ship Type (VESSEL TYPE)": "VESSEL TYPE",
    "Cause of Loss": "CAUSE OF LOSS"
}


m_1975_2000 = folium.Map(
    location=[df_periode["LATITUDE"].mean(), df_periode["LONGITUDE"].mean()],
    zoom_start=4
)

# On crée les couleurs qui seront utilisées dans la légende
def make_color_dict(values):
    unique_vals = sorted(values.unique())
    cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
    return {cat: colors.rgb2hex(cmap(i)) for i, cat in enumerate(unique_vals)}


legends = ""

# On crée les layers et les légendes
for layer_name, col in categories.items():

    layer = folium.FeatureGroup(name=layer_name, show=False)

    color_dict = make_color_dict(df_periode[col])

    # On ajoute les points selon les coordonnées
    for _, row in df_periode.iterrows():
        color = color_dict[row[col]]
        folium.CircleMarker(
            location=[row["LATITUDE"], row["LONGITUDE"]],
            radius=4,
            popup=f"<b>{row['SHIPS NAME']}</b><br>{col}: {row[col]}",
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        ).add_to(layer)

    m_1975_2000.add_child(layer)

    # On crée la légende dans une boîte en bas à gauche de la carte (c'est en html)
    legend_html = f"""
    <div id="legend_{layer_name.replace(' ', '_')}"
         style="display:none;
                position: fixed;
                bottom: 30px;
                left: 30px;
                z-index:9999;
                background-color: white;
                border:2px solid grey;
                padding:10px;
                font-size:13px;">
    <b>{layer_name}</b><br>
    """

    for cat, col_hex in color_dict.items():
        legend_html += f"""
        <i style="background:{col_hex};
                  width:12px;height:12px;
                  float:left;margin-right:6px;"></i>{cat}<br>
        """

    legend_html += "</div>"
    legends += legend_html

# Ajouter les légendes à la carte
m_1975_2000.get_root().html.add_child(folium.Element(legends))

# Script pour que ce soit la bonne légende qui s'affiche en fonction du layer
# le script est en java (ça marche car folium se base sur leaflet qui est une bibliothèque de java)
js = """
<script>

function hideLegends() {
    document.querySelectorAll('[id^=legend_]').forEach(el => {
        el.style.display = "none";
    });
}

// Exécuter au chargement
hideLegends();

// Intercepter les clics dans le LayerControl
document.addEventListener("change", function(e) {
    if (e.target.classList.contains("leaflet-control-layers-selector")) {

        hideLegends();

        if (e.target.checked) {
            // retrouver le nom du layer dans le label voisin
            let label = e.target.nextSibling.innerText.trim();
            let id = "legend_" + label.replaceAll(" ", "_");
            let legendEl = document.getElementById(id);
            if (legendEl) { legendEl.style.display = "block"; }
        }
    }
});

</script>
"""

m_1975_2000.get_root().html.add_child(folium.Element(js))

# Ajouter LayerControl
folium.LayerControl().add_to(m_1975_2000)

m_1975_2000


/tmp/ipython-input-4023818934.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
/tmp/ipython-input-4023818934.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
/tmp/ipython-input-4023818934.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', l

In [39]:
m_1975_2000.save("carte_1975_2000.html")

## 2000-2025

In [40]:
# On sélectionne les dates
periode = "2000–2025"
df_periode = df[df["PERIODE"] == periode].copy()

# On définit les catégories qu'on voudra afficher dans les différents layers
categories = {
    "Materials (CONSTRUCTION)": "CONSTRUCTION",
    "Ship Type (VESSEL TYPE)": "VESSEL TYPE",
    "Cause of Loss": "CAUSE OF LOSS"
}


m_2000_2025 = folium.Map(
    location=[df_periode["LATITUDE"].mean(), df_periode["LONGITUDE"].mean()],
    zoom_start=4
)

# On crée les couleurs qui seront utilisées dans la légende
def make_color_dict(values):
    unique_vals = sorted(values.unique())
    cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
    return {cat: colors.rgb2hex(cmap(i)) for i, cat in enumerate(unique_vals)}


legends = ""

# On crée les layers et les légendes
for layer_name, col in categories.items():

    layer = folium.FeatureGroup(name=layer_name, show=False)

    color_dict = make_color_dict(df_periode[col])

    # On ajoute les points selon les coordonnées
    for _, row in df_periode.iterrows():
        color = color_dict[row[col]]
        folium.CircleMarker(
            location=[row["LATITUDE"], row["LONGITUDE"]],
            radius=4,
            popup=f"<b>{row['SHIPS NAME']}</b><br>{col}: {row[col]}",
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        ).add_to(layer)

    m_2000_2025.add_child(layer)

    # On crée la légende dans une boîte en bas à gauche de la carte (c'est en html)
    legend_html = f"""
    <div id="legend_{layer_name.replace(' ', '_')}"
         style="display:none;
                position: fixed;
                bottom: 30px;
                left: 30px;
                z-index:9999;
                background-color: white;
                border:2px solid grey;
                padding:10px;
                font-size:13px;">
    <b>{layer_name}</b><br>
    """

    for cat, col_hex in color_dict.items():
        legend_html += f"""
        <i style="background:{col_hex};
                  width:12px;height:12px;
                  float:left;margin-right:6px;"></i>{cat}<br>
        """

    legend_html += "</div>"
    legends += legend_html

# Ajouter les légendes à la carte
m_2000_2025.get_root().html.add_child(folium.Element(legends))

# Script pour que ce soit la bonne légende qui s'affiche en fonction du layer
# le script est en java (ça marche car folium se base sur leaflet qui est une bibliothèque de java)
js = """
<script>

function hideLegends() {
    document.querySelectorAll('[id^=legend_]').forEach(el => {
        el.style.display = "none";
    });
}

// Exécuter au chargement
hideLegends();

// Intercepter les clics dans le LayerControl
document.addEventListener("change", function(e) {
    if (e.target.classList.contains("leaflet-control-layers-selector")) {

        hideLegends();

        if (e.target.checked) {
            // retrouver le nom du layer dans le label voisin
            let label = e.target.nextSibling.innerText.trim();
            let id = "legend_" + label.replaceAll(" ", "_");
            let legendEl = document.getElementById(id);
            if (legendEl) { legendEl.style.display = "block"; }
        }
    }
});

</script>
"""

m_2000_2025.get_root().html.add_child(folium.Element(js))

# Ajouter LayerControl
folium.LayerControl().add_to(m_2000_2025)

m_2000_2025


/tmp/ipython-input-874239417.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
/tmp/ipython-input-874239417.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
/tmp/ipython-input-874239417.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(

In [41]:
m_2000_2025.save("carte_2000_2025.html")

## WWI

In [42]:
# On sélectionne les dates
periode = "WWI (1914–1918)"
df_periode = df[df["PERIODE"] == periode].copy()

# On définit les catégories qu'on voudra afficher dans les différents layers
categories = {
    "Materials (CONSTRUCTION)": "CONSTRUCTION",
    "Ship Type (VESSEL TYPE)": "VESSEL TYPE",
    "Cause of Loss": "CAUSE OF LOSS"
}


m_WWI = folium.Map(
    location=[df_periode["LATITUDE"].mean(), df_periode["LONGITUDE"].mean()],
    zoom_start=4
)

# On crée les couleurs qui seront utilisées dans la légende
def make_color_dict(values):
    unique_vals = sorted(values.unique())
    cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
    return {cat: colors.rgb2hex(cmap(i)) for i, cat in enumerate(unique_vals)}


legends = ""

# On crée les layers et les légendes
for layer_name, col in categories.items():

    layer = folium.FeatureGroup(name=layer_name, show=False)

    color_dict = make_color_dict(df_periode[col])

    # On ajoute les points selon les coordonnées
    for _, row in df_periode.iterrows():
        color = color_dict[row[col]]
        folium.CircleMarker(
            location=[row["LATITUDE"], row["LONGITUDE"]],
            radius=4,
            popup=f"<b>{row['SHIPS NAME']}</b><br>{col}: {row[col]}",
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        ).add_to(layer)

    m_WWI.add_child(layer)

    # On crée la légende dans une boîte en bas à gauche de la carte (c'est en html)
    legend_html = f"""
    <div id="legend_{layer_name.replace(' ', '_')}"
         style="display:none;
                position: fixed;
                bottom: 30px;
                left: 30px;
                z-index:9999;
                background-color: white;
                border:2px solid grey;
                padding:10px;
                font-size:13px;">
    <b>{layer_name}</b><br>
    """

    for cat, col_hex in color_dict.items():
        legend_html += f"""
        <i style="background:{col_hex};
                  width:12px;height:12px;
                  float:left;margin-right:6px;"></i>{cat}<br>
        """

    legend_html += "</div>"
    legends += legend_html

# Ajouter les légendes à la carte
m_WWI.get_root().html.add_child(folium.Element(legends))

# Script pour que ce soit la bonne légende qui s'affiche en fonction du layer
# le script est en java (ça marche car folium se base sur leaflet qui est une bibliothèque de java)
js = """
<script>

function hideLegends() {
    document.querySelectorAll('[id^=legend_]').forEach(el => {
        el.style.display = "none";
    });
}

// Exécuter au chargement
hideLegends();

// Intercepter les clics dans le LayerControl
document.addEventListener("change", function(e) {
    if (e.target.classList.contains("leaflet-control-layers-selector")) {

        hideLegends();

        if (e.target.checked) {
            // retrouver le nom du layer dans le label voisin
            let label = e.target.nextSibling.innerText.trim();
            let id = "legend_" + label.replaceAll(" ", "_");
            let legendEl = document.getElementById(id);
            if (legendEl) { legendEl.style.display = "block"; }
        }
    }
});

</script>
"""

m_WWI.get_root().html.add_child(folium.Element(js))

# Ajouter LayerControl
folium.LayerControl().add_to(m_WWI)

m_WWI


/tmp/ipython-input-3498939161.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
/tmp/ipython-input-3498939161.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
/tmp/ipython-input-3498939161.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', l

In [43]:
m_WWI.save("carte_WWI.html")

## WWII

In [44]:
# On sélectionne les dates
periode = "WWII (1939–1945)"
df_periode = df[df["PERIODE"] == periode].copy()

# On définit les catégories qu'on voudra afficher dans les différents layers
categories = {
    "Materials (CONSTRUCTION)": "CONSTRUCTION",
    "Ship Type (VESSEL TYPE)": "VESSEL TYPE",
    "Cause of Loss": "CAUSE OF LOSS"
}


m_WWII = folium.Map(
    location=[df_periode["LATITUDE"].mean(), df_periode["LONGITUDE"].mean()],
    zoom_start=4
)

# On crée les couleurs qui seront utilisées dans la légende
def make_color_dict(values):
    unique_vals = sorted(values.unique())
    cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
    return {cat: colors.rgb2hex(cmap(i)) for i, cat in enumerate(unique_vals)}


legends = ""

# On crée les layers et les légendes
for layer_name, col in categories.items():

    layer = folium.FeatureGroup(name=layer_name, show=False)

    color_dict = make_color_dict(df_periode[col])

    # On ajoute les points selon les coordonnées
    for _, row in df_periode.iterrows():
        color = color_dict[row[col]]
        folium.CircleMarker(
            location=[row["LATITUDE"], row["LONGITUDE"]],
            radius=4,
            popup=f"<b>{row['SHIPS NAME']}</b><br>{col}: {row[col]}",
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        ).add_to(layer)

    m_WWII.add_child(layer)

    # On crée la légende dans une boîte en bas à gauche de la carte (c'est en html)
    legend_html = f"""
    <div id="legend_{layer_name.replace(' ', '_')}"
         style="display:none;
                position: fixed;
                bottom: 30px;
                left: 30px;
                z-index:9999;
                background-color: white;
                border:2px solid grey;
                padding:10px;
                font-size:13px;">
    <b>{layer_name}</b><br>
    """

    for cat, col_hex in color_dict.items():
        legend_html += f"""
        <i style="background:{col_hex};
                  width:12px;height:12px;
                  float:left;margin-right:6px;"></i>{cat}<br>
        """

    legend_html += "</div>"
    legends += legend_html

# Ajouter les légendes à la carte
m_WWII.get_root().html.add_child(folium.Element(legends))

# Script pour que ce soit la bonne légende qui s'affiche en fonction du layer
# le script est en java (ça marche car folium se base sur leaflet qui est une bibliothèque de java)
js = """
<script>

function hideLegends() {
    document.querySelectorAll('[id^=legend_]').forEach(el => {
        el.style.display = "none";
    });
}

// Exécuter au chargement
hideLegends();

// Intercepter les clics dans le LayerControl
document.addEventListener("change", function(e) {
    if (e.target.classList.contains("leaflet-control-layers-selector")) {

        hideLegends();

        if (e.target.checked) {
            // retrouver le nom du layer dans le label voisin
            let label = e.target.nextSibling.innerText.trim();
            let id = "legend_" + label.replaceAll(" ", "_");
            let legendEl = document.getElementById(id);
            if (legendEl) { legendEl.style.display = "block"; }
        }
    }
});

</script>
"""

m_WWII.get_root().html.add_child(folium.Element(js))

# Ajouter LayerControl
folium.LayerControl().add_to(m_WWII)

m_WWII


/tmp/ipython-input-3056923727.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
/tmp/ipython-input-3056923727.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', len(unique_vals)) # On prend la cmap (map de couleur) nommé tab20
/tmp/ipython-input-3056923727.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20', l

In [45]:
m_WWII.save("carte_WWII.html")

# Analyse

En observant les cartes, on peut observer que le nombre de naufrages diminue au fil des années (seulement 5 entre 1975 et 2025 contre 18 entre 1950 et 1975).

On peut également observer que le nombre de naufrage de navire en bois diminue, cela est très probablement dû que les navires en bois ne sont plus produit (du moins beaucoup moins qu'avant) tandis que les navires en acier sont toujours produits. Il y a donc une augementation de la solidité des matériaux.

Les types de navires les plus représentés entre avant 199 et 1950 sont les schooner (goélette). Entre 1975 et 2000 c'est les bâteaux à moteur et à partir de 1975 les tanker (bateau citerne) se font plus présent. On observe donc une évolution dans la taille des navires, ils deviennent de plus en plus grand. A côté de ça on observe qu'il n'y a pas de navires de guerre lors des guerres mondiales car les données ne sont pas sur le lieu des conflits.

La cause des naufrages évolue également avec le temps, au début il y a des causes diverses comme échouage, incendie, le navire qui coule, tempête etc. Mais avec le temps on observe que la cause de naufrage qui persiste est foundered (le navire coule). Cela peut être dû à de l'eau qui s'infiltre dans le navire via des tuyaux corrodés (les tuyaux en question servent à refroidir les moteurs), un tangage trop important, une voie d'eau, fausse maneuvre etc. Les causes sont donc des erreurs humaines car il peut y avoir des erreurs lors des maneuvres et navigations ou manque de maintenance.

Ce qu'on peut conclure c'est qu'il y a moins de naufrages car les navires sont plus grands, plus résistants et plus performants. En parallèle les systèmes de navigations et météos sont plus développée ce qui évite des risques inutiles (tempêtes, se perdre etc). Et quand il y a un naufrage c'est souvent d'origine humaine.